In [1]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

In [2]:
import pandas as pd
import numpy as np 

In [7]:
train = c.data.make_base_df()
train = c.processing.transform_save_imgs(train, c.processing.resize_n(100)); train.head()


,path,Type,filetype,num_id,<lambda>_path
0,../data/train/Type_1/0.jpg,1,jpg,0,../data/processed/resize_100/train/0.jpg
1,../data/train/Type_1/10.jpg,1,jpg,10,../data/processed/resize_100/train/10.jpg
2,../data/train/Type_1/1013.jpg,1,jpg,1013,../data/processed/resize_100/train/1013.jpg
3,../data/train/Type_1/1014.jpg,1,jpg,1014,../data/processed/resize_100/train/1014.jpg
4,../data/train/Type_1/1019.jpg,1,jpg,1019,../data/processed/resize_100/train/1019.jpg


In [10]:
train = c.processing.random_forest_transform(train, '<lambda>_path', grayscale=False)

In [12]:
test = c.data.make_test_df()
test = c.processing.transform_save_imgs(test, c.processing.resize_n(100))
test = c.processing.random_forest_transform(test, '<lambda>_path', grayscale=False)

In [13]:
from sklearn.ensemble import RandomForestClassifier as RFC
forest = RFC(n_jobs=2,n_estimators=50)

np.squeeze(train['vec'].as_matrix())

#Bit of a nasty hack to get the vectors into the right form - a (888,10000) numpy array
train_vecs = np.squeeze(np.array(train['vec'].tolist()))
forest.fit(train_vecs, train['Type'].as_matrix())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=2, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [14]:
test_vecs = np.squeeze(np.array(test['vec'].tolist()))

preds_probs = forest.predict_proba(test_vecs)

In [15]:
test = c.processing.append_probabilities(test, preds_probs, forest.classes_)

In [16]:
test.head()

,path,num_id,<lambda>_path,vec,Type_1,Type_2,Type_3
0,../data/test/0.jpg,0,../data/processed/resize_100/test/0.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.10,0.74,0.16
1,../data/test/1.jpg,1,../data/processed/resize_100/test/1.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.18,0.44,0.38
2,../data/test/10.jpg,10,../data/processed/resize_100/test/10.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.00793226214484, 0...",0.14,0.52,0.34
3,../data/test/100.jpg,100,../data/processed/resize_100/test/100.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.10,0.70,0.20
4,../data/test/101.jpg,101,../data/processed/resize_100/test/101.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.14,0.56,0.30


In [18]:
c.submission.write_submission_file('output2.csv', test)